Copyright 2020 Montvieux Ltd

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import imageio
import numpy as np
import io

from stable_baselines.common.env_checker import check_env
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from plark_game import classes
from gym_plark.envs import plark_env,plark_env_guided_reward,plark_env_top_left,plark_env_sonobuoy_deployment, panther_env_reach_top

from stable_baselines.bench import Monitor

from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv

import helper 
import datetime
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [3]:
log_dir = './logs'
os.makedirs(log_dir, exist_ok=True)
env = panther_env_reach_top.PantherEnvReachTop(config_file_path='/Components/plark-game/plark_game/game_config/10x10/balanced.json',
                                               image_based=False, 
                                               normalise=True,
                                               domain_params_in_obs=False)
env = Monitor(env, log_dir)
#env = DummyVecEnv([lambda: env])
#env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=200., gamma=0.95) 
#check_env(env)

ValueError: ("Couldn't import", 'PelicanAgentBuoys.py', ' - ', ImportError("cannot import name 'PelicanAgent'",))

In [4]:
# Test that the observations are indeed normalized
# observation, reward, done, info = env.step([0])
# np.set_printoptions(suppress=True)
# np.set_printoptions(precision=3)
# observation

## Training loop

In [5]:
n_eval_episodes = 10
training_steps = 10

In [ ]:
model = PPO2('MlpPolicy', env, seed=5000)

In [ ]:
model.learn(training_steps)

# for n in range(1000):
    
#     print("****** STARTING EVALUATION *******")
#     mean_reward, n_steps = evaluate_policy(model, 
#                                            env, 
#                                            n_eval_episodes=n_eval_episodes, 
#                                            deterministic=False, 
#                                            render=False, 
#                                            callback=None, 
#                                            reward_threshold=None, 
#                                            return_episode_rewards=False)
#     print("****** EVALUATION FINISHED *******")
#     print("Mean Reward is " + str(mean_reward))
#     print("Number of steps is " + str(n_steps))

In [ ]:
print("****** STARTING EVALUATION *******")
mean_reward, n_steps = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes, deterministic=False, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
print("****** EVALUATION FINISHED *******")
print("Mean Reward is " + str(mean_reward))
print("Number of steps is " + str(n_steps))

In [ ]:
#Save model 
basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

basepath = '/data/agents/models'
exp_name = 'test_' + basicdate
exp_path = os.path.join(basepath, exp_name)

print(exp_path)


In [ ]:

modeltype = 'PPO2'
modelplayer = env.driving_agent 
render_height = env.render_height
render_width = env.render_width
image_based = False
helper.save_model(exp_path,model,modeltype,modelplayer,render_height,render_width,image_based,basicdate)

# making the video

In [ ]:
video_path = '/testpanthernoIMAGE.mp4'
basewidth,hsize = helper.make_video(model,env,video_path)

In [ ]:
video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))